In [1]:
#  Importing modules from TensorFlow and Keras
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

2021-11-27 13:53:00.094429: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 13:53:00.094450: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import deepchem as dc
import numpy as np
from pathlib import Path
import argparse

from scipy import stats

from rdkit import DataStructs, Chem
from rdkit.Chem import AllChem
from rdkit.ML.Cluster import Butina

def describe(mols):
    """
    Function from DeepSCAMs approach

    Parameters
    ----------
    mols: list of mol-files

    Returns
    -------
    descrs: list with descriptor
            space for molecules
    """
    featurizer1 = dc.feat.CircularFingerprint()
    X1 = featurizer1([m for m in mols])
    featurizer2 = dc.feat.RDKitDescriptors()
    X2 = featurizer2([m for m in mols])
    X = np.hstack((X1, X2))
    return X

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('SCAMS_filtered.csv')

In [5]:
data['Smiles String']

0           FC(F)(F)S(=O)(=O)c1ccc(cc1)S(=O)(=O)C(F)(F)F
1               N\N=C/1\C(=O)N(CC(=O)Nc2ccccc2)c3ccccc13
2      COc1ccc2O\C(=N\NC(=O)c3ccc(Br)cc3)\C(=Cc2c1)C(...
3                      COc1ccc2C=C(C(=O)Oc2c1)c3nnc(N)s3
4         COc1cccc(OC)c1C(=O)NC2=C(C)N(C)N(C2=O)c3ccccc3
                             ...                        
911    CN1C(=O)c2cc(sc2c3ccccc13)C(=O)N4CCN(CC4)c5ccc...
912             CCCCNC(=O)CCCCCN1C(=S)Nc2cc3OCOc3cc2C1=O
913    CC1CCCC(C)N1CCCNC(=O)C2N(Cc3ccc(C)cc3)C(=O)c4c...
914                      CCC(C)N1CC(CC1=O)C(=O)NC2CCCCC2
915      CCCCN(C)C(=O)c1sc2ncnc(N3CCN(CC3)c4ccccc4)c2c1C
Name: Smiles String, Length: 916, dtype: object

In [6]:
mol_obj = [Chem.MolFromSmiles(s) for s in data['Smiles String']]

In [7]:
descriptors = describe(mol_obj)

In [8]:
descriptors.shape

(916, 2256)

In [9]:
Y = np.array(data['agg?'])

In [10]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,

In [15]:
def create_keras_model(shape=2256,
                       dropout=0.2):
    model = Sequential()
    model.add(Dense(800, input_dim=shape,
                    kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=["AUC"])
    return model



In [12]:

class Validation:
    def __init__(self, model, X, Y, name):
        self.model = model
        self.X = X
        self.Y = Y
        self.name = name
        self.results = None
        self.run()

    @staticmethod
    def f_one_mcc_score(model, X_test, Y_test):
        y_pred = model.predict(X_test)
        f_one = f1_score(Y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
        return f_one, (tp * tn - fp * fn) / ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5

    def validate(self, model, X_test, Y_test):
        test_predicted = model.predict_proba(X_test)
        auc, (lb_auc, ub_auc) = calc_auc_ci(Y_test, test_predicted[:, 1])
        f_one, mcc = self.f_one_mcc_score(model, X_test, Y_test)
        accuracy = accuracy_score(Y_test, self.model.predict(X_test))
        performance_stats = [lb_auc, auc, ub_auc, accuracy, f_one, mcc]
        return performance_stats

    def run(self):
        test_stats = self.validate(self.model, self.X, self.Y)
        results = pd.DataFrame([test_stats], index=[self.name],
                                    columns=['AUC lower estimate', 'AUC',
                                             'AUC upper estimate', 'accuracy',
                                             'F1', 'MCC'])
        self.results= results


In [13]:
model1 = create_keras_model()

2021-11-27 13:53:48.221545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-27 13:53:48.221579: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-27 13:53:48.221598: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (km): /proc/driver/nvidia/version does not exist
2021-11-27 13:53:48.221866: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
model1.fit(descriptors, Y, epochs=100)

Epoch 1/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3091 - auc: 0.9188
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3381 - auc: 0.8963
Epoch 3/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3425 - auc: 0.8976
Epoch 4/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3560 - auc: 0.8727
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3176 - auc: 0.9037
Epoch 6/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3447 - auc: 0.8950
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3333 - auc: 0.8958
Epoch 8/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3071 - auc: 0.9127
Epoch 9/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3565 - auc: 0.8864
Epoch 10/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3137 - auc: 0.9060
Epoch 11/100
29/29 [=============================

29/29 [==============================] - 0s 8ms/step - loss: 0.2850 - auc: 0.9275
Epoch 88/100
29/29 [==============================] - 0s 8ms/step - loss: 0.3426 - auc: 0.9068
Epoch 89/100
29/29 [==============================] - 0s 8ms/step - loss: 0.3753 - auc: 0.8801
Epoch 90/100
29/29 [==============================] - 0s 8ms/step - loss: 0.2977 - auc: 0.9231
Epoch 91/100
29/29 [==============================] - 0s 10ms/step - loss: 0.2982 - auc: 0.9234
Epoch 92/100
29/29 [==============================] - 0s 16ms/step - loss: 0.2931 - auc: 0.9255
Epoch 93/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2629 - auc: 0.9351
Epoch 94/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2756 - auc: 0.9217
Epoch 95/100
29/29 [==============================] - 0s 11ms/step - loss: 0.3250 - auc: 0.9070
Epoch 96/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3216 - auc: 0.9098
Epoch 97/100
29/29 [==============================] - 0s 

In [24]:
from numpy.random import seed
seed(1)

In [26]:
model1.fit(descriptors, Y, epochs=100)

Epoch 1/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2717 - auc: 0.9430
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3645 - auc: 0.9055
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2893 - auc: 0.9366
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2499 - auc: 0.9510
Epoch 5/100
29/29 [==============================] - 0s 17ms/step - loss: 0.2521 - auc: 0.9566
Epoch 6/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2392 - auc: 0.9560
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2842 - auc: 0.9409
Epoch 8/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2425 - auc: 0.9529
Epoch 9/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2269 - auc: 0.9592
Epoch 10/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2026 - auc: 0.9631
Epoch 11/100
29/29 [=============================

29/29 [==============================] - 0s 13ms/step - loss: 0.2514 - auc: 0.9489
Epoch 87/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2140 - auc: 0.9576
Epoch 88/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2083 - auc: 0.9667
Epoch 89/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2276 - auc: 0.9602
Epoch 90/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1983 - auc: 0.9663
Epoch 91/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2036 - auc: 0.9688
Epoch 92/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1557 - auc: 0.9746
Epoch 93/100
29/29 [==============================] - 1s 19ms/step - loss: 0.1931 - auc: 0.9660
Epoch 94/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2067 - auc: 0.9634
Epoch 95/100
29/29 [==============================] - 0s 13ms/step - loss: 0.1947 - auc: 0.9678
Epoch 96/100
29/29 [==============================] -

In [25]:
model1.fit(descriptors, Y, epochs=100)

Epoch 1/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2417 - auc: 0.9561
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3135 - auc: 0.9325
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2635 - auc: 0.9436
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2449 - auc: 0.9551
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2435 - auc: 0.9540
Epoch 6/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2359 - auc: 0.9550
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2468 - auc: 0.9543
Epoch 8/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2378 - auc: 0.9564
Epoch 9/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2362 - auc: 0.9568
Epoch 10/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2564 - auc: 0.9513
Epoch 11/100
29/29 [=============================

29/29 [==============================] - 0s 13ms/step - loss: 0.2128 - auc: 0.9641
Epoch 87/100
29/29 [==============================] - 0s 17ms/step - loss: 0.2103 - auc: 0.9631
Epoch 88/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2249 - auc: 0.9479
Epoch 89/100
29/29 [==============================] - 1s 19ms/step - loss: 0.2880 - auc: 0.9327
Epoch 90/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2584 - auc: 0.9496
Epoch 91/100
29/29 [==============================] - 0s 15ms/step - loss: 0.2385 - auc: 0.9587
Epoch 92/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2945 - auc: 0.9378
Epoch 93/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2509 - auc: 0.9521
Epoch 94/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2425 - auc: 0.9524
Epoch 95/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2518 - auc: 0.9514
Epoch 96/100
29/29 [==============================] -

In [20]:
val = Validation(model1, descriptors, Y, 'test')

IndexError: index 1 is out of bounds for axis 1 with size 1

In [27]:
from tensorflow.random import set_seed


In [19]:
import deepchem as dc
import numpy as np
from pathlib import Path
import argparse

from scipy import stats

from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Cluster import Butina

def describe(mols):
    """
    Function from DeepSCAMs approach

    Parameters
    ----------
    mols: list of mol-files

    Returns
    -------
    descrs: list with descriptor
            space for molecules
    """
    featurizer1 = dc.feat.CircularFingerprint()
    X1 = featurizer1([m for m in mols])
    featurizer2 = dc.feat.RDKitDescriptors()
    X2 = featurizer2([m for m in mols])
    X = np.hstack((X1, X2))
    return X


def dataset_to_splitter(splitter, dataset, type):
    """
    Feed the dataset to a chosen splitter
    """

    splitted = splitter(dataset.X, dataset.Y,
                        dataset.SMILES,
                        dataset.dataset.mols,
                        dataset.ID_name, type, 0.3)
    return splitted


def butina_cluster(mol_list, cutoff=0.35):
    """
    Reference
    __________
    @article{butina1999unsupervised,
      title={Unsupervised data base clustering based on daylight's fingerprint and Tanimoto similarity:
      A fast and automated way to cluster small and large data sets},
      author={Butina, Darko},
      journal={Journal of Chemical Information and Computer Sciences},
      volume={39},
      number={4},
      pages={747--750},
      year={1999},
      publisher={ACS Publications}
    }
    Make a clusters based Butina clustering algorithm using MorganFingerprints similarity

    Function adopted from:
    https://github.com/PatWalters/Learning_Cheminformatics/blob/master/clustering.ipynb

    Parameters
    __________
    :param mol_list: list of rdkit.Chem.rdchem.Mol object to cluster
    :param cutoff: Tanimoto similarity cutoff

    Return
    _______
    :return: list with cluster id for every molecule from mol_list

    """
    fp_list = [AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=2048) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        dists.extend([1 - x for x in sims])
    mol_clusters = Butina.ClusterData(dists, nfps, cutoff, isDistData=True)
    cluster_id_list = [0] * nfps
    for idx, cluster in enumerate(mol_clusters, 1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list

def generate_scaffolds(dataset):
    """Returns all scaffolds from the dataset.
    Adapted from deepchem
    Parameters
    ----------
    dataset: Dataset
      Dataset to be split.
    log_every_n: int, optional (default 1000)
      Controls the logger by dictating how often logger outputs
      will be produced.
    Returns
    -------
    scaffold_sets: List[List[int]]
      List of indices of each scaffold in the dataset.
    """
    scaffolds = {}
    data_len = dataset.shape[0]

    for ind, smiles in enumerate(dataset['Smiles String']):
        scaffold = _generate_scaffold(smiles)
        if scaffold not in scaffolds:
            scaffolds[scaffold] = [ind]
        else:
            scaffolds[scaffold].append(ind)

    # Sort from largest to smallest scaffold sets
    scaffolds = {key: sorted(value) for key, value in scaffolds.items()}
    scaffold_sets = [
        scaffold_set for (scaffold, scaffold_set) in sorted(
            scaffolds.items(), key=lambda x: (len(x[1]), x[1][0]), reverse=True)
    ]
    return scaffold_sets

def _generate_scaffold(smiles, include_chirality=False):
    """
    I copied the function from deepchem.
    Compute the Bemis-Murcko scaffold for a SMILES string.
    Bemis-Murcko scaffolds are described in DOI: 10.1021/jm9602928.
    They are essentially that part of the molecule consisting of
    rings and the linker atoms between them.

    Paramters
    ---------
    smiles: str
    SMILES
    include_chirality: bool, default False
    Whether to include chirality in scaffolds or not.
    Returns
    -------
    str
    The MurckScaffold SMILES from the original SMILES
    References
    ----------
    .. [1] Bemis, Guy W., and Mark A. Murcko. "The properties of known drugs.
     1. Molecular frameworks." Journal of medicinal chemistry 39.15 (1996): 2887-2893.
    Note
    ----
    This function requires RDKit to be installed.: bool
    """
    try:
        from rdkit import Chem
        from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
    except ModuleNotFoundError:
        raise ImportError("This function requires RDKit to be installed.")

    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffoldSmiles(mol=mol, includeChirality=include_chirality)
    return scaffold

def str2bool(v):
    """
    Transfer string to bool
    Parameter
    ----------
    v: str
    """
    if isinstance(v, bool):
       return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def rm_tree(pth):
    """
    Recursively remove files from the directory

    Parameter
    ----------
    pth: Pathlib path

    """
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

def file_doesnot_exist(path, file_name):
    """
    Returns full path if file exist, else raises error

    Parameters
    ----------
    file_name: str, file name
    path: str, directory

    Returns
    -------
    full_file_path: Path object

    """

    full_file_path = Path(path) / file_name
    if not full_file_path.is_file():
        raise FileNotFoundError(
            'File {} not found in location {}. Please, inter valid path and file name'.format(file_name, path))
    else:
        return full_file_path

def sampl(X_train, Y_train, sampling):
    """Computes midranks.
    Args:
       X_train
          np.array, training features
       Y_train
          np.array, responses
       sampling
          trained sampling approach

    Returns:
       X_train
          transformed np.array with training features
       Y_train
          transformed np.array with responses
    """
    sl = sampling
    X_train, Y_train = sl.fit_resample(X_train, Y_train)
    return X_train, Y_train


def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5 * (i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2

def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov

def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

def calc_auc_ci(y_true, y_pred, alpha=0.95):
    auc, auc_cov = delong_roc_variance(y_true, y_pred)
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci = stats.norm.ppf(
        lower_upper_q,
        loc=auc,
        scale=auc_std)

    ci[ci > 1] = 1
    return auc, ci


In [37]:
res = model1.predict_proba(descriptors)

In [41]:
calc_auc_ci(Y, res[:, 0])

/tmp/ipykernel_73777/1105936811.py:309: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tx = np.empty([k, m], dtype=np.float)
/tmp/ipykernel_73777/1105936811.py:310: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ty = np.empty([k, n], dtype=np.float)
/tmp/ipykernel_73777/1105936811.py:311: DeprecationWarning: `np.float` is a deprecated alias

(0.9799521366725089, array([0.97310156, 0.98680272]))

In [40]:
res[:, 1]

IndexError: index 1 is out of bounds for axis 1 with size 1